In [2]:
!pip install keras scikit-learn Pillow numpy

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.4 kB ? eta -:--:--
     -------------------------------------- 60.4/60.4 kB 809.0 kB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.3 MB 3.2 MB/s eta 0:00:03
    --------------------------------------- 0.1/9.3 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.3/9.3 MB 2.2 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/9.3 MB 3.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.3 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.3 MB 3.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.3 MB 4.1 MB/s eta 0:00:02
   -------- ----------------------------

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation
import tensorflow as tf

In [3]:
# model
model = Sequential()
# input is 150x150x3
model.add(Conv2D(150, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(150, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1540, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2220, activation='relu'))
model.add(Dense(1240, activation='linear'))
model.add(Dropout(0.1))
model.add(Dense(1044, activation='relu'))
model.add(Dense(2, activation='linear'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
# load adata
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split

In [5]:
# load data
data = []
labels = []

for file in os.listdir("data"):
    if file.endswith(".jpg"):
        try:
            img = Image.open("data/" + file)
            img = img.resize((150, 150))
            data.append(np.array(img))

            # open file that ends with .txt instead
            file = file.replace(".jpg", ".txt")
            with open("data/" + file) as f:
                label = f.readlines()
                # divide first by 90
                # divide second by 180
                label = [float(x) for x in label]
                labels.append(label)
        except:
            pass

In [6]:
print(len(data))
print(len(labels))
print(labels[0])

4447
4447
[46.77324295043945, 23.59245491027832]


In [7]:
# split data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [8]:
print(len(y_train))
print(len(X_test))

3557
890


In [12]:
# train
model.fit(np.array(X_train), np.array(y_train), batch_size=150, epochs=100, verbose=1)

Epoch 1/100
24/24 [==============================] - 8s 315ms/step - loss: 61.1840 - accuracy: 1.0000
Epoch 2/100
24/24 [==============================] - 8s 323ms/step - loss: 455.2918 - accuracy: 0.9893
Epoch 3/100
24/24 [==============================] - 7s 293ms/step - loss: 862.9434 - accuracy: 0.9727
Epoch 4/100
24/24 [==============================] - 6s 253ms/step - loss: 309.7217 - accuracy: 0.9980
Epoch 5/100
24/24 [==============================] - 6s 247ms/step - loss: 176.2312 - accuracy: 0.9980
Epoch 6/100
24/24 [==============================] - 6s 250ms/step - loss: 114.7464 - accuracy: 0.9980
Epoch 7/100
24/24 [==============================] - 8s 335ms/step - loss: 94.1765 - accuracy: 0.9992
Epoch 8/100
24/24 [==============================] - 7s 315ms/step - loss: 90.6099 - accuracy: 0.9992
Epoch 9/100
24/24 [==============================] - 8s 336ms/step - loss: 84.2714 - accuracy: 0.9994
Epoch 10/100
24/24 [==============================] - 8s 321ms/step - loss: 5

In [11]:
# evaluate
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(score)

28/28 [==============================] - 1s 24ms/step - loss: 728.0364 - accuracy: 0.9989
[728.0364379882812, 0.9988763928413391]


In [ ]:
# save model
model.save('model.h5')